# Import library

In [1]:
import random
import math
import copy
from collections import defaultdict
import pandas as pd
import numpy as np
from tqdm import trange
import matplotlib.pyplot as plt

# Dummy data: mata kuliah

In [2]:
# Kolom: code, name, program, priority (1=highest), room_type ("lab" atau "class"), lecturer_id, duration (slots)
courses = pd.DataFrame([
    ("CIE61101","Perkembangan Peserta Didik","Pendidikan Teknologi Informasi",1,'class',39,2),
    ("CIE61102","Perkembangan Peserta Didik","Pendidikan Teknologi Informasi",2,'class',59,1),
    ("CIE61102","Filsafat Pendidikan dan Sains","Pendidikan Teknologi Informasi",1,'class',103,1),
    ("CIE61103","Filsafat Pendidikan dan Sains","Pendidikan Teknologi Informasi",2,'class',103,1),
    ("CIE61103","Komunikasi dan Teknologi Pendidikan","Pendidikan Teknologi Informasi",1,'class',103,2),
    ("CIE61104","Komunikasi dan Teknologi Pendidikan","Pendidikan Teknologi Informasi",1,'class',39,1),
    ("CIE61108","Penilaian Hasil Belajar","Pendidikan Teknologi Informasi",1,'class',100,1),
    ("CIE61109","Pembelajaran Mikro","Pendidikan Teknologi Informasi",1,'class',39,2),
    ("CIE61110","Pembelajaran Mikro","Pendidikan Teknologi Informasi",1,'class',100,1),
    ("CIE61110","Perancangan Pembelajaran","Pendidikan Teknologi Informasi",1,'class',39,1),
    ("CIE61111","Pengenalan Lapangan Persekolahan 1","Pendidikan Teknologi Informasi",1,'class',103,2),
    ("CIE61112","Praktikum Jaringan","Pendidikan Teknologi Informasi",1,'lab',41,3),
    ("CIE61113","Praktikum Sistem Operasi","Pendidikan Teknologi Informasi",2,'lab',41,2),
])


courses.index.name = 'course_id'

# Dummy data: Ruangan

In [3]:
rooms = pd.DataFrame([
    ("F11.1","Gedung F FILKOM",11,"R. Kuliah FILKOM","class"),
    ("F11.3","Gedung F FILKOM",11,"R. Kuliah FILKOM","class"),
    ("F2.1","Gedung F FILKOM",2,"R. Kuliah (Smart Class)","class"),
    ("F2.2","Gedung F FILKOM",2,"R. Kuliah (Smart Class)","class"),
    ("LABJAR1","Gedung Lab A",1,"Laboratorium Jaringan","lab"),
    ("LABJAR2","Gedung Lab B",2,"Laboratorium Jaringan","lab"),
    ("LABOS1","Gedung Lab C",3,"Laboratorium OS","lab"),
], columns=['kode_ruang','lokasi','lantai','keterangan','room_type'])
rooms.index = rooms['kode_ruang']

# Timeslot generation

In [4]:
# We'll discretize into hourly slots starting at 7 to 17 (inclusive start), each slot is 1 hour.
# Hard constraints define break windows which we'll remove as unavailable slots.
DAYS = ['Mon','Tue','Wed','Thu','Fri']
HOURS = list(range(7,18)) # 7..17


# Remove break slots (approximation):
# Mon-Thu break 12:10-12:30 -> remove hour 12
# Fri break 11.15-12.30 -> remove hours 11 and 12
available_slots = []
for day in DAYS:
    for h in HOURS:
        if day in ['Mon','Tue','Wed','Thu'] and h==12:
            continue
        if day=='Fri' and h in (11,12):
            continue
        available_slots.append((day,h))


SLOT_INDEX = {s:i for i,s in enumerate(available_slots)}


# Helper: enumerate contiguous slots for course duration

# Lecturer time preferences (soft constraint)

In [5]:
# small example: lecturer 39 prefers morning (7-11) except friday
lecturer_prefs = {
    39: [(d,h) for d in DAYS for h in range(7,11)],
    103: [(d,h) for d in DAYS for h in range(13,17)],
}

# Fitness & constraint evaluation

In [6]:
BIG_PENALTY = 10_000

def check_hard_constraints(schedule):
    # schedule: dict course_id -> (slot_index, room_code)
    penalty = 0
    # 1. room double-book
    room_usage = defaultdict(list)
    for cid,(sidx,room) in schedule.items():
        duration = int(courses.loc[cid,6])
        for dt in range(duration):
            idx = sidx + dt
            if idx >= len(available_slots):
                penalty += BIG_PENALTY  # out of time range
            else:
                room_usage[(room,idx)].append(cid)
    for k,v in room_usage.items():
        if len(v)>1:
            penalty += BIG_PENALTY * (len(v)-1)
    # 2 & 3: lecturer not in more than one place at same time
    lecturer_usage = defaultdict(list)
    for cid,(sidx,room) in schedule.items():
        lecturer = courses.loc[cid,5]
        duration = int(courses.loc[cid,6])
        for dt in range(duration):
            idx = sidx + dt
            lecturer_usage[(lecturer,idx)].append((cid,room))
    for k,v in lecturer_usage.items():
        if len(v)>1:
            penalty += BIG_PENALTY * (len(v)-1)
    # 4. Time windows already enforced by available_slots; check slot indices within range done above
    # 5. Room type matching
    for cid,(sidx,room) in schedule.items():
        room_type = rooms.loc[room,'room_type']
        required = courses.loc[cid,4]
        if room_type != required:
            penalty += BIG_PENALTY
    return penalty


def soft_score(schedule):
    # lower is better
    score = 0
    # 1. priority: if two courses overlap same slot (allowed but soft), we prefer lower priority assigned earlier.
    # We'll add priority value for each scheduled slot (so lower priority adds less)
    for cid,(sidx,room) in schedule.items():
        pr = courses.loc[cid,3]
        score += pr
    # 2. minimize floor movement for lecturers: if a lecturer teaches in two adjacent slots with large floor diff, penalize
    lec_slots = defaultdict(list)
    for cid,(sidx,room) in schedule.items():
        lec = courses.loc[cid,5]
        lec_slots[lec].append((sidx,room))
    for lec,assigns in lec_slots.items():
        assigns_sorted = sorted(assigns)
        for i in range(len(assigns_sorted)-1):
            s1,r1 = assigns_sorted[i]
            s2,r2 = assigns_sorted[i+1]
            slot_time1 = available_slots[s1]
            slot_time2 = available_slots[s2]
            # if consecutive in time (next slot) and different floors, penalize difference
            if s2==s1+1:
                f1 = rooms.loc[r1,'lantai']
                f2 = rooms.loc[r2,'lantai']
                score += abs(f1-f2)
    # 3. lecturer preferences
    for cid,(sidx,room) in schedule.items():
        lec = courses.loc[cid,5]
        slot = available_slots[sidx]
        if lec in lecturer_prefs and slot not in lecturer_prefs[lec]:
            score += 1
    return score


def fitness(schedule):
    # fitness: lower is better
    return check_hard_constraints(schedule) + soft_score(schedule)

# Helper random initialization

In [7]:
def random_schedule():
    sch = {}
    for cid in courses.index:
        duration = int(courses.loc[cid,6])
        # pick starting slot that fits
        max_start = len(available_slots)-duration
        sidx = random.randint(0,max_start)
        # pick a room of matching type
        req = courses.loc[cid,4]
        candidate_rooms = rooms[rooms['room_type']==req].index.tolist()
        room = random.choice(candidate_rooms)
        sch[cid] = (sidx, room)
    return sch

# Simple Genetic Algorithm (baseline)

In [8]:
def ga_optimize(generations=200,pop_size=50,crossover_prob=0.8,mut_prob=0.2):
    # encode as dict per individual
    pop = [random_schedule() for _ in range(pop_size)]
    pop_scores = [fitness(ind) for ind in pop]
    best_idx = int(np.argmin(pop_scores))
    best = pop[best_idx]
    best_score = pop_scores[best_idx]
    for gen in trange(generations, desc='GA'):
        new_pop = []
        while len(new_pop)<pop_size:
            # selection: tournament
            i1,i2 = random.sample(range(pop_size),2)
            parent1 = pop[i1] if pop_scores[i1]<pop_scores[i2] else pop[i2]
            i3,i4 = random.sample(range(pop_size),2)
            parent2 = pop[i3] if pop_scores[i3]<pop_scores[i4] else pop[i4]
            # crossover
            if random.random()<crossover_prob:
                # one-point crossover over course ids order
                keys = list(courses.index)
                cx = random.randint(1,len(keys)-1)
                child = {}
                for i,k in enumerate(keys):
                    child[k] = copy.deepcopy(parent1[k]) if i<cx else copy.deepcopy(parent2[k])
            else:
                child = copy.deepcopy(parent1)
            # mutation: randomly reassign some courses
            if random.random()<mut_prob:
                m = random.choice(list(courses.index))
                duration = int(courses.loc[m,6])
                sidx = random.randint(0,len(available_slots)-duration)
                candidate_rooms = rooms[rooms['room_type']==courses.loc[m,4]].index.tolist()
                room = random.choice(candidate_rooms)
                child[m] = (sidx,room)
            new_pop.append(child)
        pop = new_pop
        pop_scores = [fitness(ind) for ind in pop]
        cur_best_idx = int(np.argmin(pop_scores))
        if pop_scores[cur_best_idx] < best_score:
            best_score = pop_scores[cur_best_idx]
            best = pop[cur_best_idx]
    return best, best_score

# Simulated Annealing

In [9]:
def sa_optimize(iterations=5000, start_temp=1000, cooling=0.995):
    current = random_schedule()
    current_score = fitness(current)
    best, best_score = current, current_score
    T = start_temp
    for i in range(iterations):
        # neighbor: change scheduling of one random course
        neighbor = copy.deepcopy(current)
        m = random.choice(list(courses.index))
        duration = int(courses.loc[m,6])
        sidx = random.randint(0,len(available_slots)-duration)
        candidate_rooms = rooms[rooms['room_type']==courses.loc[m,4]].index.tolist()
        room = random.choice(candidate_rooms)
        neighbor[m] = (sidx,room)
        ns = fitness(neighbor)
        d = ns - current_score
        if d<0 or random.random() < math.exp(-d/T):
            current, current_score = neighbor, ns
            if ns < best_score:
                best, best_score = neighbor, ns
        T *= cooling
    return best, best_score

# Particle Swarm Optimization (discrete variant)

In [10]:
def ps0_optimize(particles=40, iterations=300):
    # position: numerical vector length = 2*Ncourses where each course has slot index and room index
    n = len(courses)
    room_lists = [rooms[rooms['room_type']==courses.loc[cid,4]].index.tolist() for cid in courses.index]
    room_index_maps = [ {r:i for i,r in enumerate(lst)} for lst in room_lists]
    dim = n*2
    # initialize
    pos = []
    vel = []
    pbest = []
    pbest_score = []
    for i,cid in enumerate(courses.index):
        pass
    for p in range(particles):
        vec = []
        for idx,cid in enumerate(courses.index):
            duration = int(courses.loc[cid,6])
            sidx = random.randint(0,len(available_slots)-duration)
            rlist = room_lists[idx]
            ridx = random.randrange(len(rlist))
            vec.append(sidx)
            vec.append(ridx)
        pos.append(np.array(vec,dtype=float))
        vel.append(np.zeros_like(pos[-1]))
        # decode
        sched = decode_particle(pos[-1], room_lists)
        sc = fitness(sched)
        pbest.append(pos[-1].copy())
        pbest_score.append(sc)
    gbest_idx = int(np.argmin(pbest_score))
    gbest = pbest[gbest_idx].copy()
    gbest_score = pbest_score[gbest_idx]
    w=0.5;c1=1.5;c2=1.5
    for it in trange(iterations, desc='PSO'):
        for i in range(particles):
            r1=random.random(); r2=random.random()
            vel[i] = w*vel[i] + c1*r1*(pbest[i]-pos[i]) + c2*r2*(gbest-pos[i])
            pos[i] = pos[i] + vel[i]
            # clamp and discretize when decoding
            sched = decode_particle(pos[i], room_lists)
            sc = fitness(sched)
            if sc < pbest_score[i]:
                pbest[i] = pos[i].copy(); pbest_score[i]=sc
                if sc < gbest_score:
                    gbest = pos[i].copy(); gbest_score = sc
    return decode_particle(gbest, room_lists), gbest_score


def decode_particle(vec, room_lists):
    sched = {}
    arr = [int(round(x)) for x in vec]
    for i,cid in enumerate(courses.index):
        sidx = max(0,min(len(available_slots)-int(courses.loc[cid,6]), arr[2*i]))
        rlist = room_lists[i]
        ridx = arr[2*i+1] % len(rlist)
        room = rlist[ridx]
        sched[cid] = (sidx, room)
    return sched

# Ant Colony Optimization (very simplified)

In [11]:
def aco_optimize(ants=40, iterations=200):
    # pheromone for assigning start slot and room per course
    n = len(courses)
    pher_slot = np.ones((n,len(available_slots)))
    pher_room = [np.ones(len(rooms[rooms['room_type']==courses.loc[cid,4]])) for cid in courses.index]
    best=None; best_score=1e18
    for it in trange(iterations, desc='ACO'):
        solutions = []
        scores = []
        for a in range(ants):
            sched = {}
            for i,cid in enumerate(courses.index):
                duration = int(courses.loc[cid,6])
                feasible_slots = list(range(0,len(available_slots)-duration+1))
                # probabilistic pick based on pheromone
                ps = pher_slot[i,feasible_slots]
                probs = ps/ps.sum()
                sidx = np.random.choice(feasible_slots,p=probs)
                rlist = rooms[rooms['room_type']==courses.loc[cid,4]].index.tolist()
                pr = pher_room[i]
                probs_r = pr/pr.sum()
                ridx = np.random.choice(len(rlist),p=probs_r)
                room = rlist[ridx]
                sched[cid] = (sidx,room)
            sc = fitness(sched)
            solutions.append(sched); scores.append(sc)
            if sc < best_score:
                best, best_score = sched, sc
        # update pheromone
        pher_slot *= 0.9
        for i in range(n):
            for a,sched in enumerate(solutions):
                sidx,_ = sched[courses.index[i]]
                pher_slot[i,sidx] += 1.0/(1+scores[a])
                ridx = list(rooms[rooms['room_type']==courses.loc[courses.index[i],4]].index).index(sched[courses.index[i]][1])
                pher_room[i][ridx] += 1.0/(1+scores[a])
    return best,best_score

# Improved GA (elitism + adaptive mutation)

In [12]:
def improved_ga(generations=200,pop_size=60):
    pop = [random_schedule() for _ in range(pop_size)]
    pop_scores = [fitness(ind) for ind in pop]
    best_idx = int(np.argmin(pop_scores))
    best = pop[best_idx]; best_score = pop_scores[best_idx]
    for gen in trange(generations, desc='ImpGA'):
        new_pop = []
        # elitism: keep top 5
        elite_k = max(1,int(0.05*pop_size))
        idxs = np.argsort(pop_scores)
        for k in range(elite_k):
            new_pop.append(copy.deepcopy(pop[idxs[k]]))
        while len(new_pop)<pop_size:
            # selection roulette
            fitness_vals = np.array(pop_scores)
            # convert to positive selection probs (lower fitness -> higher chance)
            probs = (1/(1+fitness_vals))/((1/(1+fitness_vals)).sum())
            p1 = pop[np.random.choice(range(pop_size),p=probs)]
            p2 = pop[np.random.choice(range(pop_size),p=probs)]
            # crossover
            keys = list(courses.index)
            cx = random.randint(1,len(keys)-1)
            child = {}
            for i,k in enumerate(keys):
                child[k] = copy.deepcopy(p1[k]) if i<cx else copy.deepcopy(p2[k])
            # adaptive mutation based on diversity
            mut_prob = 0.1 if gen<generations*0.6 else 0.02
            if random.random()<mut_prob:
                m = random.choice(list(courses.index))
                duration = int(courses.loc[m,6])
                sidx = random.randint(0,len(available_slots)-duration)
                candidate_rooms = rooms[rooms['room_type']==courses.loc[m,4]].index.tolist()
                room = random.choice(candidate_rooms)
                child[m] = (sidx,room)
            new_pop.append(child)
        pop = new_pop
        pop_scores = [fitness(ind) for ind in pop]
        cur_best_idx = int(np.argmin(pop_scores))
        if pop_scores[cur_best_idx] < best_score:
            best_score = pop_scores[cur_best_idx]
            best = pop[cur_best_idx]
    return best,best_score

# Improved PSO (discrete with local search)

In [13]:
def improved_pso(particles=30, iterations=250):
    # use ps0 but add local hill-climb occasionally
    sched,score = ps0_optimize(particles=particles, iterations=iterations//2)
    # local hill-climb from result
    best, best_score = sched, score
    for _ in trange(200, desc='ImpPSO-local'):
        neighbor = copy.deepcopy(best)
        m = random.choice(list(courses.index))
        duration = int(courses.loc[m,6])
        sidx = random.randint(0,len(available_slots)-duration)
        candidate_rooms = rooms[rooms['room_type']==courses.loc[m,4]].index.tolist()
        room = random.choice(candidate_rooms)
        neighbor[m] = (sidx,room)
        ns = fitness(neighbor)
        if ns<best_score:
            best, best_score = neighbor, ns
    return best,best_score

# Puma Optimization Algorithm (simple implementation)

In [14]:
def puma_optimize(pop_size=30, iterations=200, stalk_prob=0.7, pounce_prob=0.3):
    """
    Puma optimizer adapted to schedule dicts used in notebook.
    - pop_size: jumlah puma (populasi)
    - iterations: iterasi maksimum
    - stalk_prob: probabilitas memilih mekanisme stalking (mendekati best)
    - pounce_prob: probabilitas lompat eksplorasi per course
    """
    # helper: random schedule already defined as random_schedule()
    # helper: room candidates per course
    room_candidates = {cid: rooms[rooms['room_type']==courses.loc[cid,4]].index.tolist()
                       for cid in courses.index}

    # initialize population (list of schedule dicts)
    population = [random_schedule() for _ in range(pop_size)]
    scores = [fitness(ind) for ind in population]

    best_idx = int(np.argmin(scores))
    best = copy.deepcopy(population[best_idx])
    best_score = scores[best_idx]

    convergence = []

    for it in trange(iterations, desc='PUMA'):
        new_pop = []

        for i in range(pop_size):
            current = copy.deepcopy(population[i])

            # STALK: create candidate by moving some courses closer to best
            stalk_candidate = copy.deepcopy(current)
            for cid in courses.index:
                if random.random() < 0.5:  # choose some courses to attempt stalking
                    # with small chance, nudge timeslot towards best's timeslot
                    cur_slot, cur_room = stalk_candidate[cid]
                    best_slot, best_room = best[cid]
                    # nudge timeslot one step towards best (if different)
                    if cur_slot < best_slot:
                        new_slot = min(cur_slot + 1, len(available_slots)-int(courses.loc[cid,6]))
                    elif cur_slot > best_slot:
                        new_slot = max(cur_slot - 1, 0)
                    else:
                        new_slot = cur_slot
                    # adopt best room with some chance
                    new_room = best_room if random.random() < 0.5 else cur_room
                    stalk_candidate[cid] = (new_slot, new_room)

            # POUNCE: exploratory candidate (random reassignments)
            pounce_candidate = copy.deepcopy(current)
            for cid in courses.index:
                if random.random() < pounce_prob:
                    duration = int(courses.loc[cid,6])
                    sidx = random.randint(0, len(available_slots)-duration)
                    rlist = room_candidates[cid]
                    room = random.choice(rlist)
                    pounce_candidate[cid] = (sidx, room)

            # choose between stalking and pouncing adaptively
            if random.random() < stalk_prob:
                candidate = stalk_candidate
            else:
                candidate = pounce_candidate

            # small local tweak: sometimes swap two courses' slots to escape local minima
            if random.random() < 0.1:
                a,b = random.sample(list(courses.index), 2)
                ca = candidate[a]
                cb = candidate[b]
                # swap only timeslot (keep room) if room types compatible
                if courses.loc[a,4] == courses.loc[b,4]:
                    candidate[a] = (cb[0], ca[1])
                    candidate[b] = (ca[0], cb[1])

            new_pop.append(candidate)

        # Evaluate new population
        new_scores = [fitness(ind) for ind in new_pop]

        # Combine and select best pop_size solutions (minimization)
        combined = population + new_pop
        combined_scores = scores + new_scores
        idxs = np.argsort(combined_scores)[:pop_size]
        population = [copy.deepcopy(combined[i]) for i in idxs]
        scores = [combined_scores[i] for i in idxs]

        # update best
        if scores[0] < best_score:
            best_score = scores[0]
            best = copy.deepcopy(population[0])

        convergence.append(best_score)

    return best, best_score, convergence

# Coati Optimization Algorithm (simple implementation)

In [15]:
def coati_optimize(pop_size=40, iterations=300):
    # Inspired by coati behaviour: group-based exploration + local search
    pop = [random_schedule() for _ in range(pop_size)]
    scores = [fitness(p) for p in pop]
    best_idx = int(np.argmin(scores)); best=pop[best_idx]; best_score=scores[best_idx]
    for it in trange(iterations, desc='COATI'):
        # sort by fitness
        order = np.argsort(scores)
        pop = [pop[i] for i in order]
        scores = [scores[i] for i in order]
        # top explorers: perform local search (intensification)
        for i in range(max(1,int(0.2*pop_size))):
            candidate = copy.deepcopy(pop[i])
            # local tweak several courses
            for _ in range(3):
                m = random.choice(list(courses.index))
                duration = int(courses.loc[m,6])
                sidx = max(0, min(len(available_slots)-duration, candidate[m][0]+random.randint(-2,2)))
                rlist = rooms[rooms['room_type']==courses.loc[m,4]].index.tolist()
                room = random.choice(rlist) if random.random()<0.3 else candidate[m][1]
                candidate[m] = (sidx,room)
            sc = fitness(candidate)
            if sc < scores[i]:
                pop[i]=candidate; scores[i]=sc
                if sc < best_score:
                    best, best_score = candidate, sc
        # others perform exploration by recombining with random individuals
        for i in range(int(0.2*pop_size), pop_size):
            donor = random.choice(pop[:max(1,int(0.3*pop_size))])
            target = copy.deepcopy(pop[i])
            # replace random subset of courses from donor
            num_replace = random.randint(1, max(1,len(courses)//4))
            keys = random.sample(list(courses.index), num_replace)
            for k in keys:
                target[k] = copy.deepcopy(donor[k])
            # random walk
            if random.random()<0.3:
                m = random.choice(list(courses.index))
                duration = int(courses.loc[m,6])
                sidx = random.randint(0,len(available_slots)-duration)
                rlist = rooms[rooms['room_type']==courses.loc[m,4]].index.tolist()
                room = random.choice(rlist)
                target[m] = (sidx,room)
            sc = fitness(target)
            pop[i]=target; scores[i]=sc
            if sc < best_score:
                best, best_score = target, sc
    return best,best_score

# Run all algorithms (short demo with small iterations)

In [17]:
algos = [
    ('GA', lambda: ga_optimize(generations=150,pop_size=50)),
    ('ImpGA', lambda: improved_ga(generations=150,pop_size=60)),
    ('SA', lambda: sa_optimize(iterations=3000)),
    ('PSO', lambda: ps0_optimize(particles=30, iterations=200)),
    ('ImpPSO', lambda: improved_pso(particles=20, iterations=200)),
    ('ACO', lambda: aco_optimize(ants=30, iterations=150)),
    ('PUMA', lambda: puma_optimize(pop_size=30, iterations=200)),
    ('COATI', lambda: coati_optimize(pop_size=30, iterations=200)),
]

results = {}
for name, fn in algos:
    print('\nRunning', name)
    if name == 'PUMA':
        best, sc, curve = fn()            # ambil 3 nilai
        results[name] = (best, sc, curve) # simpan 3
    else:
        best, sc = fn()                   # ambil 2 nilai
        results[name] = (best, sc, None)  # simpan 2 + placeholder
    print(name, 'score =', sc)


Running GA


GA: 100%|██████████| 150/150 [00:08<00:00, 17.38it/s]


GA score = 16

Running ImpGA


ImpGA: 100%|██████████| 150/150 [00:10<00:00, 14.76it/s]


ImpGA score = 16

Running SA
SA score = 16

Running PSO


...

ACO score = 17

Running PUMA


PUMA: 100%|██████████| 200/200 [00:07<00:00, 26.53it/s]


PUMA score = 16

Running COATI


COATI: 100%|██████████| 200/200 [00:07<00:00, 25.37it/s]

COATI score = 16


# Simple reporting

In [ ]:
for name,(sched,sc) in results.items():
    print('\n===',name,'score',sc,'===')
    # print schedule in human-friendly form
    rows = []
    for cid,(sidx,room) in sched.items():
        timeslot = available_slots[sidx]
        rows.append((cid, courses.loc[cid,1], courses.loc[cid,5], timeslot[0], timeslot[1], room))
    df = pd.DataFrame(rows, columns=['code','name','lecturer','day','hour','room'])
    print(df.sort_values(['day','hour']).to_string(index=False))


=== GA score 16 ===
 code                                name  lecturer day  hour    room
    5 Komunikasi dan Teknologi Pendidikan        39 Fri     9   F11.3
    3       Filsafat Pendidikan dan Sains       103 Fri    14   F11.1
    4 Komunikasi dan Teknologi Pendidikan       103 Fri    16    F2.1
    0          Perkembangan Peserta Didik        39 Mon     7   F11.3
    9            Perancangan Pembelajaran        39 Mon     9    F2.2
    1          Perkembangan Peserta Didik        59 Mon    13   F11.1
   12            Praktikum Sistem Operasi        41 Thu     8 LABJAR1
    2       Filsafat Pendidikan dan Sains       103 Thu    16   F11.1
    8                  Pembelajaran Mikro       100 Tue     7   F11.1
   11                  Praktikum Jaringan        41 Tue    10 LABJAR1
    7                  Pembelajaran Mikro        39 Wed     7   F11.3
   10  Pengenalan Lapangan Persekolahan 1       103 Wed    16    F2.1
    6             Penilaian Hasil Belajar       100 Wed    17    F2.2

# Save best (example)

In [ ]:
best_name = min(results.keys(), key=lambda k: results[k][1])
best_sched = results[best_name][0]
print('\nBest overall:', best_name, 'score=', results[best_name][1])

# You can extend and tune each algorithm: add more realistic timeslot durations (90/120 mins),
# incorporate student-group conflicts, more sophisticated local search for COATI, and
# increase iteration/pop sizes for production runs.


Best overall: GA score= 16
